In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("./artifacts/data_ingestion/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       193573 non-null  int64  
 1   carat    193573 non-null  float64
 2   cut      193573 non-null  object 
 3   color    193573 non-null  object 
 4   clarity  193573 non-null  object 
 5   depth    193573 non-null  float64
 6   table    193573 non-null  float64
 7   x        193573 non-null  float64
 8   y        193573 non-null  float64
 9   z        193573 non-null  float64
 10  price    193573 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 16.2+ MB


In [5]:
df.drop(['id'],axis=1,inplace=True)

In [6]:
x = df.drop(['price'],axis=1)
y = df[['price']]

In [7]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [8]:
categorical_cols = x.select_dtypes(include='object').columns
numerical_cols = x.select_dtypes(exclude='object').columns


In [9]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [11]:
from sklearn.impute import  SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.pipeline import  Pipeline
from sklearn.compose import ColumnTransformer

In [18]:
num_pipeline = Pipeline(steps=[('imputer',SimpleImputer(strategy='median')),('scaler',StandardScaler())])
cat_pipeline = Pipeline(steps = [('imputer',SimpleImputer(strategy='most_frequent')),('OrdinalEncoder', OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),('scaler',StandardScaler())])
preprocessor = ColumnTransformer([('num_pipeline',num_pipeline,numerical_cols),('cat_pipeline',cat_pipeline,categorical_cols)]) 

In [14]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=30)
X_train

,carat,cut,color,clarity,depth,table,x,y,z
168192,0.34,Ideal,I,VVS2,60.9,57.0,4.56,4.53,2.76
35202,0.90,Good,E,SI1,63.8,57.0,6.07,6.03,3.87
41091,1.02,Premium,G,VS1,62.7,58.0,6.35,6.39,4.00
31239,0.32,Premium,G,VS2,62.1,59.0,4.37,4.35,2.71
45722,0.35,Ideal,J,VVS2,61.1,56.0,4.53,4.57,2.78
...,...,...,...,...,...,...,...,...,...
66455,0.31,Ideal,E,SI1,61.8,56.0,4.31,4.35,2.68
46220,1.25,Ideal,G,SI2,62.0,56.0,6.88,6.95,4.28
98804,1.00,Good,G,SI1,63.5,56.0,6.29,6.37,4.02
48045,1.10,Ideal,G,VS1,59.9,60.0,6.68,6.77,4.01


In [19]:
x_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())

In [20]:
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [21]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [23]:
regression=LinearRegression()
regression.fit(x_train,y_train)

LinearRegression()

In [24]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [26]:
## Train multiple models
## Model Ecaluation
models={'LinearRegression':LinearRegression(),'Lasso':Lasso(),'Ridge':Ridge(),'Elasticnet':ElasticNet()}
trained_model_list=[]
model_list=[]
r2_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)
    #Make Predictions
    y_pred=model.predict(X_test)
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)
    r2_list.append(r2_square)
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 1013.9047094344005
MAE: 674.0255115796833
R2 score 93.68908248567512


Lasso
Model Training Performance
RMSE: 1013.8784226767013
MAE: 675.071692336216
R2 score 93.68940971841704


Ridge
Model Training Performance
RMSE: 1013.9059272771649
MAE: 674.05558007982
R2 score 93.68906732505937


Elasticnet
Model Training Performance
RMSE: 1533.416245606405
MAE: 1060.7368759154729
R2 score 85.56494831165182


